## Importaciones

In [7]:
import os
import math
import random
import numpy as np
from tqdm import tqdm
from numba import njit
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Tuple, List
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from modules.labeling_lib import get_labels_one_direction
from modules.labeling_lib import sliding_window_clustering
from modules.tester_lib import tester_one_direction
from modules.export_lib import export_model_to_ONNX
import warnings
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [8]:
# Obtener precios
def get_prices(hyper_params) -> pd.DataFrame:
    history_file = os.path.join(hyper_params["history_path"], f"{hyper_params['symbol']}_{hyper_params['timeframe']}.csv")
    p = pd.read_csv(history_file, sep=r"\s+")
    pFixed = pd.DataFrame(columns=['time', 'close'])
    pFixed['time'] = p['<DATE>'] + ' ' + p['<TIME>']
    pFixed['time'] = pd.to_datetime(pFixed['time'], format='mixed')
    pFixed['close'] = p['<CLOSE>']
    pFixed.set_index('time', inplace=True)
    return pFixed.dropna()
# Ingeniería de características
@njit
def compute_features(close, periods, periods_meta, stats):
    n = len(close)
    total_features = (len(periods) * len(stats)) + len(periods_meta)
    features = np.full((n, total_features), np.nan)

    def std_manual(x):
        m = np.mean(x)
        return np.sqrt(np.sum((x - m) ** 2) / (x.size - 1))

    def skew_manual(x):
        m = np.mean(x)
        s = std_manual(x)
        return np.mean(((x - m) / s) ** 3) if s != 0 else 0.0

    def kurt_manual(x):
        m = np.mean(x)
        s = std_manual(x)
        return np.mean(((x - m) / s) ** 4) - 3 if s != 0 else 0.0
    
    def zscore_manual(x):
        m = np.mean(x)
        s = std_manual(x)
        return (x[0] - m) / s if s != 0 else 0.0
    
    def entropy_manual(x):
        bins = 10
        minv = np.min(x)
        maxv = np.max(x)
        width = (maxv - minv) / bins
        if width == 0:
            return 0.0
        hist = np.zeros(bins)
        for val in x:
            idx = int((val - minv) / width)
            if idx == bins:  # caso borde
                idx -= 1
            hist[idx] += 1
        total = x.size
        entropy = 0.0
        for i in range(bins):
            p = hist[i] / total
            if p > 0:
                entropy -= p * np.log(p)
        return entropy

    def slope_manual(x):
        n = x.size
        x_idx = np.ascontiguousarray(np.arange(n))
        x_mean = np.mean(x_idx)
        y_mean = np.mean(x)
        numerator = np.sum((x_idx - x_mean) * (x - y_mean))
        denominator = np.sum((x_idx - x_mean) ** 2)
        return numerator / denominator if denominator != 0 else 0.0

    def momentum_manual(x):
        return (x[0] / x[-1]) - 1.0 if x[-1] != 0 else 0.0
    
    def roc_manual(x):
        n = len(x)
        if n < 2:
            return 0.0
        return ((x[0] - x[-1]) / x[-1]) * 100 if x[-1] != 0 else 0.0
    
    def fractal_dimension_manual(x):
        # Método de box-counting simplificado
        x = np.ascontiguousarray(x)  # Asegurar array contiguo
        eps = np.std(x) / 4  # epsilon como fracción de std
        if eps == 0:  # Evitar división por cero
            return 1.0
        count = np.sum(np.abs(np.diff(x)) > eps)
        if count == 0:
            return 1.0
        return 1.0 + np.log(count) / np.log(len(x))
    
    def hurst_manual(x):
        # Exponente de Hurst simplificado
        n = len(x)
        if n < 4:
            return 0.5
        lags = min(n-1, 20)  # usar máximo 20 lags
        tau = np.ascontiguousarray(np.arange(1, lags+1))
        rs = np.zeros(lags)
        
        for lag in range(1, lags+1):
            roll_mean = np.mean(x[:n-lag+1])
            roll_std = std_manual(x[:n-lag+1])
            if roll_std == 0:
                continue
            rs[lag-1] = np.max(x[:n-lag+1]) / roll_std
        
        valid_rs = rs[rs != 0]
        if len(valid_rs) < 2:
            return 0.5
        
        return np.mean(np.log(valid_rs)) / np.log(n) if n > 1 else 0.5

    # Procesar períodos normales
    col = 0
    for win in periods:
        for s in stats:
            for i in range(win, n):
                window = close[i - win:i][::-1]
                if s == "std":
                    features[i, col] = std_manual(window)
                elif s == "skew":
                    features[i, col] = skew_manual(window)
                elif s == "kurt":
                    features[i, col] = kurt_manual(window)
                elif s == "zscore":
                    features[i, col] = zscore_manual(window)
                elif s == "mean":
                    features[i, col] = np.mean(window)
                elif s == "range":
                    features[i, col] = np.max(window) - np.min(window)
                elif s == "median":
                    features[i, col] = np.median(window)
                elif s == "mad":
                    features[i, col] = np.mean(np.abs(window - np.mean(window)))
                elif s == "var":
                    features[i, col] = np.var(window)
                elif s == "entropy":
                    features[i, col] = entropy_manual(window)
                elif s == "slope":
                    features[i, col] = slope_manual(window)
                elif s == "momentum":
                    features[i, col] = momentum_manual(window)
                elif s == "roc":
                    features[i, col] = roc_manual(window)
                elif s == "fractal":
                    features[i, col] = fractal_dimension_manual(window)
                elif s == "hurst":
                    features[i, col] = hurst_manual(window)
            col += 1  # Incrementar col después de procesar todas las filas para esta estadística y ventana

    # Procesar períodos meta
    for win in periods_meta:
        for i in range(win, n):
            window = close[i - win:i][::-1]
            features[i, col] = std_manual(window)
        col += 1

    return features

def get_features(data: pd.DataFrame, hp):
    close = data['close'].values
    index = data.index
    periods = hp["periods"]
    periods_meta = hp["periods_meta"]
    stats = hp["stats"]
    if len(stats) == 0:
        raise ValueError("La lista de estadísticas está vacía.")
    # Asegurar que los arrays sean contiguos
    close = np.ascontiguousarray(close)
    periods = np.ascontiguousarray(periods)
    periods_meta = np.ascontiguousarray(periods_meta)
    feats = compute_features(close, periods, periods_meta, stats)
    if np.isnan(feats).all():
        return pd.DataFrame(index=index)
    # Nombres de columnas
    colnames = []
    for p in periods:
        for s in stats:
            colnames.extend([f"{p}_{s}_feature"])
    for p in periods_meta:
        colnames.extend([f"{p}_std_meta_feature"])
    df = pd.DataFrame(feats, columns=colnames, index=index)
    df["close"] = data["close"]
    return df.dropna()
    
def test_model_one_direction(
        dataset: pd.DataFrame,
        result:  list,
        forward: datetime,
        backward: datetime,
        markup:  float,
        direction: str,
        plt: bool = False):

    pr_tst = dataset.copy()
    X = pr_tst.drop(columns=['close'])
    X_meta = X.loc[:,  X.columns.str.contains('meta_feature')]
    X      = X.loc[:, ~X.columns.str.contains('meta_feature')]

    pr_tst['labels']      = result[0].predict_proba(X)[:,1]
    pr_tst['meta_labels'] = result[1].predict_proba(X_meta)[:,1]

    # Corrección aquí:
    pr_tst[['labels', 'meta_labels']] = (pr_tst[['labels', 'meta_labels']] > 0.5).astype(float)

    return tester_one_direction(pr_tst, forward, backward, markup, direction, plt)

## Main

In [9]:
def fit_final_models(clustered: pd.DataFrame,
                     meta: pd.DataFrame,
                     oos_data: pd.DataFrame,
                     hp: Dict[str, Any]) -> Tuple[float, Any, Any]:
    """Entrena modelo principal + meta‑modelo y evalúa en OOS.

    Devuelve (R2, model, meta_model).
    """
    # ---------- 1) main model ----------
    X_main = clustered.drop(columns=['labels', *meta.columns[meta.columns.str.contains('_meta_feature')]])
    y_main = clustered['labels'].astype('int16')

    # ---------- 2) meta‑model ----------
    X_meta = meta.loc[:, meta.columns.str.contains('_meta_feature')]
    y_meta = meta['clusters'].astype('int16')
    # 3) Split aleatorio (70/30)
    train_X, test_X, train_y, test_y = train_test_split(
        X_main, y_main, train_size=0.7, shuffle=True)
    train_X_m, test_X_m, train_y_m, test_y_m = train_test_split(
        X_meta, y_meta, train_size=0.7, shuffle=True)
    # debug
    # common_index = X_main.index[0]
    # display(X_main.loc[[common_index]])
    # display(X_meta.loc[[common_index]])
    # 4) Hiper‑parámetros CatBoost (con valores por defecto + overrides)
    cat_main_params = dict(
        iterations=hp.get('cat_main_iterations', 500),
        depth=hp.get('cat_main_depth', 6),
        learning_rate=hp.get('cat_main_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_main_l2_leaf_reg', 3.0),
        custom_loss=['Accuracy'],
        eval_metric='Accuracy',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    model = CatBoostClassifier(**cat_main_params)
    model.fit(train_X, train_y, eval_set=(test_X, test_y), early_stopping_rounds=25)

    cat_meta_params = dict(
        iterations=hp.get('cat_meta_iterations', 500),
        depth=hp.get('cat_meta_depth', 6),
        learning_rate=hp.get('cat_meta_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_meta_l2_leaf_reg', 3.0),
        custom_loss=['F1'],
        eval_metric='F1',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    meta_model = CatBoostClassifier(**cat_meta_params)
    meta_model.fit(train_X_m, train_y_m, eval_set=(test_X_m, test_y_m), early_stopping_rounds=15)

    # 5) Evaluación en datos fuera de muestra
    R2 = test_model_one_direction(
        oos_data,
        [model, meta_model],
        hp['full forward'],
        hp['forward'],
        hp['markup'],
        hp['direction'],
        plt=False,
    )
    if math.isnan(R2):
        R2 = -1.0
    return R2, model, meta_model

# ----------------------------------------------------------------------------
#      ─── FUNCIÓN OBJETIVO PARA OPTUNA ───
# ----------------------------------------------------------------------------

def objective(trial: optuna.trial.Trial, base_hp: Dict[str, Any], study=None) -> float:
    hp = base_hp.copy()

    # µ··· Espacio de búsqueda optimizado ···µ
    # Parámetros de clustering más amplios para encontrar patrones más diversos
    hp['n_clusters'] = trial.suggest_int('n_clusters', 10, 100, step=10)  
    hp['window_size'] = trial.suggest_int('window_size', 200, 800, step=50)
    
    # Parámetros de etiquetado más agresivos
    hp['label_min'] = trial.suggest_int('label_min', 2, 8)
    hp['label_max'] = trial.suggest_int('label_max', hp['label_min']+8, 40)
    hp['markup'] = trial.suggest_float("markup", 0.1, 0.4)

    # CatBoost principal - Mayor capacidad de aprendizaje
    hp['cat_main_iterations'] = trial.suggest_int('cat_main_iterations', 300, 2000, step=100)
    hp['cat_main_depth'] = trial.suggest_int('cat_main_depth', 6, 12)
    hp['cat_main_learning_rate'] = trial.suggest_float('cat_main_learning_rate', 0.005, 0.4, log=True)
    hp['cat_main_l2_leaf_reg'] = trial.suggest_float('cat_main_l2_leaf_reg', 0.5, 10.0)

    # CatBoost meta - Enfoque en precisión
    hp['cat_meta_iterations'] = trial.suggest_int('cat_meta_iterations', 200, 1000, step=100)
    hp['cat_meta_depth'] = trial.suggest_int('cat_meta_depth', 5, 10)
    hp['cat_meta_learning_rate'] = trial.suggest_float('cat_meta_learning_rate', 0.01, 0.3, log=True)
    hp['cat_meta_l2_leaf_reg'] = trial.suggest_float('cat_meta_l2_leaf_reg', 0.5, 8.0)

    # Optimización de períodos para el modelo principal
    n_periods_main = trial.suggest_int('n_periods_main', 5, 15)
    main_periods = []
    for i in range(n_periods_main):
        period = trial.suggest_int(f'period_main_{i}', 2, 200, log=True)
        main_periods.append(period)
    main_periods = sorted(list(set(main_periods)))  # Eliminar duplicados y ordenar
    if len(main_periods) < 3:  # Asegurar mínimo de períodos
        return -np.inf
    hp['periods'] = main_periods

    # Optimización de períodos para el meta-modelo
    n_periods_meta = 1 #trial.suggest_int('n_periods_meta', 1, 2)
    meta_periods = []
    for i in range(n_periods_meta):
        period = trial.suggest_int(f'period_meta_{i}', 2, 5)
        meta_periods.append(period)
    meta_periods = sorted(list(set(meta_periods)))  # Eliminar duplicados y ordenar
    hp['periods_meta'] = meta_periods

    # Selección de estadísticas con pesos
    stat_choices = {
        "std": 0.8,
        "skew": 0.6, 
        "kurt": 0.5,
        "zscore": 0.9,
        "mean": 0.7,
        "range": 0.8,
        "median": 0.6,
        "mad": 0.5,
        "var": 0.7,
        "entropy": 0.4,
        "slope": 0.9,
        "momentum": 0.8,
        "roc": 0.7,
        "fractal": 0.6,
        "hurst": 0.5
    }
    
    selected_stats = []
    for stat, weight in stat_choices.items():
        if trial.suggest_float(f"stat_weight_{stat}", 0, 1) > (1 - weight):
            selected_stats.append(stat)
            
    if len(selected_stats) < 3:  # Asegurar mínimo de features
        return -np.inf
    hp["stats"] = selected_stats

    # Dataset completo
    full_ds = get_features(get_prices(hp), hp)
    ds_train = full_ds[(full_ds.index > hp['backward']) & (full_ds.index < hp['forward'])]
    ds_oos = full_ds[(full_ds.index >= hp['forward']) & (full_ds.index < hp['full forward'])]
    
    # Clustering con ventana deslizante
    data = sliding_window_clustering(
        ds_train,
        n_clusters=hp['n_clusters'],
        window_size=hp['window_size']
    )
    
    best_R2 = -math.inf
    valid_clusters = 0
    
    # Evaluar clusters ordenados por tamaño
    cluster_sizes = data['clusters'].value_counts()
    for clust in cluster_sizes.index:
        clustered_data = data[data['clusters'] == clust].copy()
        if len(clustered_data) < 750:  # Aumentar mínimo de muestras
            continue
            
        valid_clusters += 1
        clustered_data = get_labels_one_direction(
            clustered_data,
            markup=hp['markup'],
            min=hp['label_min'],
            max=hp['label_max'],
            direction=hp['direction'])

        clustered_data = clustered_data.drop(['close', 'clusters'], axis=1)
        meta_data = data.copy()
        meta_data['clusters'] = (meta_data['clusters'] == clust).astype(int)

        R2, model, meta_model = fit_final_models(
            clustered_data,
            meta_data.drop(['close'], axis=1),
            ds_oos,
            hp
        )

        if R2 > best_R2:
            best_R2 = R2
            best_pack = (model, meta_model)
            
            if study is not None:
                prev_best = study.user_attrs.get("best_r2", -np.inf)
                if best_R2 > prev_best:
                    study.set_user_attr("best_model", best_pack[0])
                    study.set_user_attr("best_meta_model", best_pack[1])
                    study.set_user_attr("best_r2", best_R2)
                    study.set_user_attr("best_stats", hp["stats"])
                    study.set_user_attr("best_periods", hp["periods"])
                    study.set_user_attr("best_periods_meta", hp["periods_meta"])
                    
    # Penalizar si muy pocos clusters válidos
    if valid_clusters < 3:
        best_R2 *= 0.5

    return best_R2

# ----------------------------------------------------------------------------
#                 ─── PIPELINE DE OPTIMIZACIÓN + EXPORT ───
# ----------------------------------------------------------------------------

def optimize_and_export(symbol, timeframe, model_number, n_trials):
    """Lanza Optuna, guarda el mejor modelo y lo exporta a ONNX."""

    common_file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
    mql5_files_folder = r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/'
    mql5_include_folder = r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Include/ajmtrz/include/Dmitrievsky'

    base_hp: Dict[str, Any] = {
        'symbol': symbol,
        'timeframe': timeframe,
        'models_export_path': mql5_files_folder,
        'include_export_path': mql5_include_folder,
        'history_path': common_file_folder,
        'best_models': [],
        'stats': [],
        'model_number': model_number,
        'markup': 0.20,
        'label_min'  : 1,
        'label_max'  : 15,
        'direction': 'buy',
        'n_clusters': 30,
        'window_size': 350,
        'periods': [i for i in range(5, 300, 30)],
        'periods_meta': [5],
        'backward': datetime(2020, 3, 26),
        'forward': datetime(2024, 1, 1),
        'full forward': datetime(2026, 1, 1),
    }

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda t: objective(t, base_hp, study), n_trials=n_trials, show_progress_bar=False)


    print("\n┌───────────────────────────────────────────────┐")
    print("│      MEJOR RESULTADO = {:.4f}                 │".format(study.best_value))
    print("└───────────────────────────────────────────────┘\n")
    print("Parámetros óptimos:\n", study.best_params)

    # Recuperar el mejor modelo y meta‑modelo
    base_hp.update(study.best_params)
    model      = study.user_attrs["best_model"]
    meta_model = study.user_attrs["best_meta_model"]
    best_r2    = study.user_attrs["best_r2"]
    base_hp['stats'] = study.user_attrs["best_stats"]
    base_hp['periods'] = study.user_attrs["best_periods"]
    base_hp['periods_meta'] = study.user_attrs["best_periods_meta"]
    base_hp.pop('best_models', None)
    print("Exportando modelos ONNX… R2 = {:.4f}".format(best_r2))
    export_model_to_ONNX(best_models=[model, meta_model], **base_hp)

if __name__ == "__main__":
    symbol = 'XAUUSD'
    timeframe = 'H1'
    n_trials_per_model = 1
    model_range = range(0, 1)
    for i in tqdm(model_range, desc=f"Optimizando {symbol}/{timeframe}", unit="modelo"):
        try:
            optimize_and_export(symbol, timeframe, i, n_trials=n_trials_per_model)
        except Exception as e:
            tqdm.write(f"\nError procesando modelo {i}: {e}") # tqdm.write evita romper la barra
            continue
    print("Proceso de optimización completado.")

Optimizando XAUUSD/H1: 100%|██████████| 1/1 [00:10<00:00, 10.84s/modelo]


┌───────────────────────────────────────────────┐
│      MEJOR RESULTADO = 2.1900                 │
└───────────────────────────────────────────────┘

Parámetros óptimos:
 {'n_clusters': 30, 'window_size': 700, 'label_min': 2, 'label_max': 18, 'markup': 0.21490689395601428, 'cat_main_iterations': 800, 'cat_main_depth': 10, 'cat_main_learning_rate': 0.010399841441761907, 'cat_main_l2_leaf_reg': 5.708021659059657, 'cat_meta_iterations': 200, 'cat_meta_depth': 8, 'cat_meta_learning_rate': 0.2052901480911591, 'cat_meta_l2_leaf_reg': 0.896674438708679, 'n_periods_main': 6, 'period_main_0': 22, 'period_main_1': 98, 'period_main_2': 129, 'period_main_3': 11, 'period_main_4': 3, 'period_main_5': 199, 'n_periods_meta': 2, 'period_meta_0': 3, 'period_meta_1': 4, 'stat_weight_std': 0.7178274533239766, 'stat_weight_skew': 0.16556769161617357, 'stat_weight_kurt': 0.2979526331352256, 'stat_weight_zscore': 0.1431887083041441, 'stat_weight_mean': 0.11784705942036189, 'stat_weight_range': 0.5431651190